This notebook requires: 

* astropy.nddata updates from [astropy/astropy#14175](https://github.com/astropy/astropy/pull/14175)
* glue-astronomy updates from [glue-viz/glue-astronomy#81](https://github.com/glue-viz/glue-astronomy/pull/81)

In [ ]:
import warnings
import tempfile

import numpy as np
from specutils import Spectrum1D
from astroquery.mast import Observations
from astropy.nddata import NDDataArray
from astropy.nddata.nduncertainty import StdDevUncertainty
from regions import PixCoord, CirclePixelRegion

from jdaviz import Cubeviz

cubeviz = Cubeviz()

data_dir = tempfile.gettempdir()

fn = "jw02732-o004_t004_miri_ch1-shortmediumlong_s3d.fits"
result = Observations.download_file(f"mast:JWST/product/{fn}", local_path=f'{data_dir}/{fn}')

with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    cubeviz.load_data(f'{data_dir}/{fn}')

cubeviz.show()

In [ ]:
def collapse_cube(
    # supply the helper:
    cubeviz,
    # choose your collapse operation (sum, mean, max, min):
    statistic, 
    # you can specify the axes you want to collapse:
    axis=None, 
    # or alternatively you can collapse over all spatial/spectral dims:
    spectral=None,
    spatial=None,
    # provide a spatial subset to operate on:
    subset=None,
    # these keys specify which `Data` to extract from collection:
    flux_ext='SCI', 
    uncert_ext='ERR',
    # add data? make data visible?
    add_data=True,
    show_in_viewer=True,
):
    """
    Currently implemented without spatial subset masking, 
    so `nddata.mask` will be None for now. We can generalize this function
    for spatial subset masks by swapping `Data.get_object` for 
    `Data.get_subset_object`.
    """
    supported_statistics = ['sum', 'mean', 'min', 'max']
    
    if statistic.lower() not in supported_statistics:
        raise ValueError(f"Collapse statistic '{statistic}' not supported")
    
    # find index within data collection which correspond to the
    # flux cube with an extension like "[SCI]"
    [dc_flux_index] = [
        i for i, d in enumerate(cubeviz.app.data_collection) 
        if d.label.endswith(f"[{flux_ext}]")
    ]
    # similarly, find index for an extension like "[ERR]"
    [dc_uncert_index] = [
        i for i, d in enumerate(cubeviz.app.data_collection) 
        if d.label.endswith(f"[{uncert_ext}]")
    ]
    
    # rely on the glue-astronomy translators implemented 
    # in https://github.com/glue-viz/glue-astronomy/pull/81
    # and use the mask defined by `subset` if there is one:
    if subset is None:
        nddata = cubeviz.app.data_collection[dc_flux_index].get_object(NDDataArray)
    else: 
        nddata = cubeviz.app.data_collection[dc_flux_index].get_subset_object(subset, NDDataArray)

    uncertainty = cubeviz.app.data_collection[dc_uncert_index].get_object(StdDevUncertainty)
    nddata = NDDataArray(nddata, uncertainty=uncertainty)

    all_axes = tuple(i for i in range(nddata.meta['NAXIS']))
    dispersion_axis = nddata.meta['DISPAXIS']
    spatial_axes = tuple(i for i in all_axes if i != dispersion_axis)
    
    if axis is None:
        if spectral and spatial:
            axis = all_axes
        elif spectral:
            axis = dispersion_axis
        elif spatial:
            axis = spatial_axes
        else: 
            raise ValueError("Must specify either `axis`, `spectral=True`, or `spatial=True`.")

    collapsed = getattr(nddata, statistic)(axis=axis)

    if add_data:
        if spectral:
            # Spectral collapse produces a spatial image. Compute (+show) both the 
            # collapsed NDDataArray in the flux-viewer and uncert-viewer 
            flux_label = f'spectral collapse ({statistic})'
            uncert_label = f'spectral collapse ({statistic})'
            cubeviz.app.add_data(
                collapsed, flux_label
            )
            cubeviz.app.add_data(
                collapsed.uncertainty, uncert_label
            )
            if show_in_viewer: 
                cubeviz.app.add_data_to_viewer(
                    cubeviz._default_flux_viewer_reference_name,
                    flux_label
                )
                cubeviz.app.add_data_to_viewer(
                    cubeviz._default_uncert_viewer_reference_name,
                    uncert_label
                )

        elif spatial:
            # Spatial collapse produces a spectrum. We need to provide
            # a spectral axis since the shape of the resulting NDDataArray
            # `collapsed` will be different from the shape of the inherited
            # WCS object

            # `dispersion_axis` is zero-based index like numpy, but the 
            # FITS standard for axis labels is one-based.
            pxls = np.arange(nddata.meta[f'NAXIS{dispersion_axis+1}'])
            spectral_axis = nddata.wcs.spectral_wcs.pixel_to_world(pxls)
            
            # convert the NDData to Spectrum1D manually, since
            # NDData objects are not presently supported by 
            # specutils (https://github.com/astropy/specutils/pull/1003)
            spec = Spectrum1D(
                flux=collapsed.data << collapsed.unit, 
                uncertainty=collapsed.uncertainty, 
                spectral_axis=spectral_axis, 
                mask=collapsed.mask, 
                wcs=collapsed.wcs
            )
            # hack from Kyle to make any collapsed spectra visible
            # in the Data selection menu (top left of spectrum viewer)
            spec.meta['Plugin'] = ""
            
            data_label = f"spatial collapse ({statistic})"
            cubeviz.app.add_data(
                spec, data_label
            )
            cubeviz.app.add_data_to_viewer(
                cubeviz._default_spectrum_viewer_reference_name, 
                data_label, visible=show_in_viewer
            )

            # reset the x/y axis limits to include all visible data:
            spectrum_viewer = cubeviz.app.get_viewer(
                cubeviz._default_spectrum_viewer_reference_name
            )
            spectrum_viewer.state.reset_limits()

    return collapsed

You can now collapse a cube along a given set of dimensions like so:

In [ ]:
nddata = collapse_cube(cubeviz, 'sum', spatial=True)

The kwarg name I chose here is `spatial=True` for collapses along the spatial dimensions (produces a spectrum), and `spectral=True` for collapses on the spectral dimension (produces an image). Happy to debate if this is a sensible choice.

Now let's select a spatial (or spectral!) subset, before running another collapse on the spatial dimensions (within the subset) to a spectrum. 

In [ ]:
# add a spatial subset through the API:
circ = CirclePixelRegion(center=PixCoord(x=20, y=25), radius=5)
cubeviz.load_regions([circ])

In this implementation, I haven't yet added logic for simultaneously selecting both spectral and spatial subsets, but that logic will be straightforward to add. 

Now just specify within which subset you want to collapse:

In [ ]:
nddata2 = collapse_cube(cubeviz, 'mean', spatial=True, subset='Subset 1')

And look, we have uncertainties:

In [ ]:
nddata2.uncertainty